In [1]:
from config import config

import dataexplorer
dataexploreragent = dataexplorer.DataExplorerEscargot(config, model_name="azuregpt-4o-mini")

import cyphergpt
cyphergptAgent = cyphergpt.CypherEscargot(config, node_types = "BiologicalProcess, BodyPart, CellularComponent, Datatype, Disease, Drug, DrugClass, Gene, MolecularFunction, Pathway, Symptom", relationship_types = """CHEMICALBINDSGENE
    CHEMICALDECREASESEXPRESSION
    CHEMICALINCREASESEXPRESSION
    DRUGINCLASS
    DRUGCAUSESEFFECT
    DRUGTREATSDISEASE
    GENEPARTICIPATESINBIOLOGICALPROCESS
    GENEINPATHWAY
    GENEINTERACTSWITHGENE
    GENEHASMOLECULARFUNCTION
    GENEASSOCIATEDWITHCELLULARCOMPONENT
    GENEASSOCIATESWITHDISEASE
    SYMPTOMMANIFESTATIONOFDISEASE
    BODYPARTUNDEREXPRESSESGENE
    BODYPARTOVEREXPRESSESGENE
    DISEASELOCALIZESTOANATOMY
    DISEASEASSOCIATESWITHDISEASET""",
    relationship_scores="""CHEMICALBINDSGENE	['affinity_nM']
    CHEMICALDECREASESEXPRESSION	['z_score']
    CHEMICALINCREASESEXPRESSION	['z_score']
    DISEASELOCALIZESTOANATOMY	['p_fisher']
    GENEASSOCIATESWITHDISEASE	['score']
    GENECOVARIESWITHGENE	['correlation']
    SYMPTOMMANIFESTATIONOFDISEASE	['p_fisher']
    TRANSCRIPTIONFACTORINTERACTSWITHGENE	['confidence']""" ,
    model_name="azuregpt-4o-mini")
cyphergptAgent.memgraph_client.schema = """Node properties are the following:
Node name: 'BiologicalProcess', Node properties: ['commonName']
Node name: 'BodyPart', Node properties: ['commonName']
Node name: 'CellularComponent', Node properties: ['commonName']
Node name: 'Disease', Node properties: ['commonName']
Node name: 'Drug', Node properties: ['commonName']
Node name: 'DrugClass', Node properties: ['commonName']
Node name: 'Gene', Node properties: ['commonName', 'geneSymbol', 'typeOfGene']
Node name: 'MolecularFunction', Node properties: ['commonName']
Node name: 'Pathway', Node properties: ['pathwayName']
Node name: 'Symptom', Node properties: ['commonName']
Relationship properties are the following:
The relationships are the following:
(:Drug)-[:CHEMICALBINDSGENE]-(:Gene)
(:Drug)-[:CHEMICALDECREASESEXPRESSION]-(:Gene)
(:Drug)-[:CHEMICALINCREASESEXPRESSION]-(:Gene)
(:Drug)-[:DRUGINCLASS]-(:DrugClass)
(:Drug)-[:DRUGCAUSESEFFECT]-(:Disease)
(:Drug)-[:DRUGTREATSDISEASE]-(:Disease)
(:Gene)-[:GENEPARTICIPATESINBIOLOGICALPROCESS]-(:BiologicalProcess)
(:Gene)-[:GENEINPATHWAY]-(:Pathway)
(:Gene)-[:GENEINTERACTSWITHGENE]-(:Gene)
(:Gene)-[:GENEHASMOLECULARFUNCTION]-(:MolecularFunction)
(:Gene)-[:GENEASSOCIATEDWITHCELLULARCOMPONENT]-(:CellularComponent)
(:Gene)-[:GENEASSOCIATESWITHDISEASE]-(:Disease)
(:Symptom)-[:SYMPTOMMANIFESTATIONOFDISEASE]-(:Disease)
(:BodyPart)-[:BODYPARTUNDEREXPRESSESGENE]-(:Gene)
(:BodyPart)-[:BODYPARTOVEREXPRESSESGENE]-(:Gene)
(:Disease)-[:DISEASELOCALIZESTOANATOMY]-(:BodyPart)
(:Disease)-[:DISEASEASSOCIATESWITHDISEASET]-(:Disease)"""

In [2]:
from escargot.multiagent.multiagent_manager import MultiAgentManager
agents = {
    "DataExplorerAgent": {"agent": dataexploreragent, "role": "Able to code and debug in Python. Can use pandas, numpy, scikit-learn, and matplotlib. Has access to the original dataframe, which is a gene dataset, where all the columns are gene symbols. These gene symbols can be extracted by using the CypherGPT knowledge extraction. This agent has no domain knowledge on biomedical data and must request the CypherGPT instead. Returns data visualizations and dataframes."},
    "CypherGPT": {"agent": cyphergptAgent, "role": "Able to execute Cypher queries from a biomedical knowledge graph. When using this agent, in the plan you provide, put into quotes the entire unchanged instructions as it can handle complex queries. Do not rephrase or alter the question and include nothing else in the step's description as this will be fed directly to the agent. If using, use only ONCE in the plan and it MUST be the first step. Other agents can handle the results afterwards. Returns lists and dictionaries."},
}
manager = MultiAgentManager(config, agents, chat_id = 0)

MultiAgentManager initialized with the following agents:
DataExplorerAgent
CypherGPT
Chat ID: 0
Message ID: 0
Current chat history:
{
  "0": {
    "agent": "CypherGPT",
    "message": "Find pathways related to Alzheimer and filter genes connected to both these pathways and Alzheimer's disease. Then, find and rank the drugs linked to these genes based on the number of connections.",
    "result": [
      {
        "drug_name": "Zinc acetate",
        "gene_count": 12
      },
      {
        "drug_name": "Zinc chloride",
        "gene_count": 12
      },
      {
        "drug_name": "Zinc",
        "gene_count": 11
      },
      {
        "drug_name": "Menadione",
        "gene_count": 10
      },
      {
        "drug_name": "Copper",
        "gene_count": 9
      },
      {
        "drug_name": "Varenicline",
        "gene_count": 9
      },
      {
        "drug_name": "Crizotinib",
        "gene_count": 9
      },
      {
        "drug_name": "Doxorubicin",
        "gene_count": 8


In [3]:
result0 = manager.direct_chat("""@CypherGPT Find pathways related to Alzheimer and filter genes connected to both these pathways and Alzheimer's disease. Then, find and rank the drugs linked to these genes based on the number of connections.""", answer_type="array", message_id = 0)

Running escargot...
Running CypherGPT...


2024-12-10 04:04:09,110 - cyphergpt.py - get_knowledge(340) - ```escargot|SHOW```Executed Cypher: 
MATCH (d:Disease {commonName: "Alzheimer's Disease"})-[:GENEASSOCIATESWITHDISEASE]-(g:Gene)-[:GENEINPATHWAY]-(p:Pathway)
WITH DISTINCT p
MATCH (p)-[:GENEINPATHWAY]-(g:Gene)-[:GENEASSOCIATESWITHDISEASE]-(d)
WITH DISTINCT g
MATCH (d2:Drug)-[:CHEMICALBINDSGENE|:CHEMICALDECREASESEXPRESSION|:CHEMICALINCREASESEXPRESSION]-(g)
RETURN d2.commonName AS drug_name, count(DISTINCT g) AS gene_count
ORDER BY gene_count DESC

2024-12-10 04:04:11,563 - cyphergpt.py - ask(454) - Output: {'result': [{'drug_name': 'Zinc acetate', 'gene_count': 12}, {'drug_name': 'Zinc chloride', 'gene_count': 12}, {'drug_name': 'Zinc', 'gene_count': 11}, {'drug_name': 'Menadione', 'gene_count': 10}, {'drug_name': 'Copper', 'gene_count': 9}, {'drug_name': 'Varenicline', 'gene_count': 9}, {'drug_name': 'Crizotinib', 'gene_count': 9}, {'drug_name': 'Doxorubicin', 'gene_count': 8}, {'drug_name': 'Irinotecan', 'gene_count': 8}, {

In [4]:
result1 = manager.direct_chat("""@DataExplorerAgent Which of the drugs are in this list: ['Clozapine', 'Adenosine', 'Aripiprazole', 'Pioglitazone', 'Risperidone', 'Sunitinib', 'Tamoxifen', 'Vandetanib', 'Verapamil', 'Vorinostat']""", message_id = 1)

Running escargot...
Running DataExplorerAgent...


<string>:4: ResourceWarning: unclosed file <_io.BufferedReader name='checkpoint-0.pkl'>
2024-12-10 04:37:07,790 - dataexplorer.py - ask(409) - Output: The code performs the following steps:

1. It imports the necessary libraries, `dill` and `pandas`.
2. It loads data from a file named `checkpoint-0.pkl` using `dill.load`.
3. It defines a list of specific drugs to check for.
4. It creates a pandas DataFrame from the loaded data.
5. It filters the DataFrame to include only the rows where the `drug_name` is in the `specific_drugs` list.
6. It saves the filtered DataFrame to a CSV file named `filtered_drugs.csv`.

New files created in the working directory:
- `filtered_drugs.csv`

The CSV file `filtered_drugs.csv` contains the filtered data with the following entries:

```
drug_name,gene_count
Tamoxifen,5
Vorinostat,5
Sunitinib,4
Pioglitazone,3
Vandetanib,2
Clozapine,2
Risperidone,1
Aripiprazole,1
Verapamil,1
```
